In [1]:
import pandas as pd
import os
from kan import *
# create a KAN: 2D inputs, 1D output, and 5 hidden neurons. cubic spline (k=3), 5 grid intervals (grid=5).
model = KAN(width=[2,5,1], grid=5, k=3, seed=0)

import sys
sys.set_int_max_str_digits(0)

In [2]:
question_numbers = [1, 2, 3, 4, 5, 6, 7, 8]         # Numbers of questions from DASS to run through
target = "anxiety_status"
models_to_train = 10        # Number of models for each number of questions from DASS
models_per_question = 10   # Number of ensembles per model
test_split = 0.1    
seed = 42
random.seed(seed)

data_folder = "./data"
models_folder = "./models"

feats_df = pd.read_csv(os.path.join(data_folder, "features.csv"))
labels_df = pd.read_csv(os.path.join(data_folder, "labels.csv"))

questions = [20, 9, 30, 11, 19, 2, 36, 28, 4, 23]


In [3]:
for num_questions in question_numbers:
    lst_comb =[]
    
    train_losses = []
    test_losses = []
    if num_questions == 2:
        break
    
    
    for a in range(models_to_train):
        cols = ["gender_m", "gender_f", "region_other", 
                    "region_east", "region_west", "age_norm"]
        if num_questions == 1:  # Only has 1 question
            if a >= len(questions):
                break
            question_nums = [questions[a]]  # a=0, question[0]=20, question_nums contains a list of question numbers
            
        else:  # More than 1 question
            question_nums = random.sample(questions, num_questions)  # Randomly sample num_questions from all questions
            question_nums.sort() # Sort the questions
            #resample if already in list
            while question_nums in lst_comb:
                question_nums = random.sample(questions, num_questions)
            lst_comb.append(question_nums)
        
        for q in question_nums:  # q is one of the selected questions
            for j in range(4):
                cols.append("Q{0}A_{1}".format(q, j))  # Generate the question numbers
        features = feats_df[cols]  # Get the features for the selected questions
        

        labels = labels_df[[target]].copy()
        print("Num Questions: ", num_questions)
        print("Selected questions: ", question_nums, "a: ", a)
        print("features: ", len(features.columns))
        
        np.random.seed(seed)
        shufId = np.random.permutation(int(len(labels)))
        index = int(test_split * len(labels.index))  # Index of data to be used for testing

        df_prist = features.iloc[shufId[0:index]]   # Data for testing
        df_trainvalid = features.iloc[shufId[index:-1]] # Data for training and validation

        gt_prist = labels.iloc[shufId[0:index]]     # Labels for testing
        gt_trainvalid = labels.iloc[shufId[index:-1]]       # Labels for training and validation

        for b in range(models_per_question):
            np.random.seed(b)
            shufId = np.random.permutation(int(len(gt_trainvalid)))
            index = int((1/9) * len(gt_trainvalid.index))

            df_valid = df_trainvalid.iloc[shufId[0:index]]
            df_train = df_trainvalid.iloc[shufId[index:-1]]

            gt_valid = gt_trainvalid.iloc[shufId[0:index]]
            gt_train = gt_trainvalid.iloc[shufId[index:-1]]

            df_valid = df_valid.reset_index(drop=True)
            df_train = df_train.reset_index(drop=True)

            gt_valid = gt_valid.reset_index(drop=True)
            gt_train = gt_train.reset_index(drop=True)

            dataset = {}
            # Convert data to PyTorch tensors and store in dataset
            dataset['train_input'] = torch.from_numpy(df_train.values).float()
            dataset['train_label'] = torch.from_numpy(gt_train.values).float()
            dataset['test_input'] = torch.from_numpy(df_prist.values).float()
            dataset['test_label'] = torch.from_numpy(gt_prist.values).float()
            print("b: ", b)
            print("Train Input Size:", dataset['train_input'].shape)
            print("Train Label Size:", dataset['train_label'].shape)
            model = KAN(width=[dataset['train_input'].size(1),5,1], grid=2, k=3, seed=0)
            
            model(dataset['train_input'])  # obtain activation
            lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs']
            model.auto_symbolic(lib=lib)
            
            def train_acc(): 
                return torch.mean((torch.round(model(dataset['train_input'])[:,0]) == dataset['train_label'][:,0]).float())

            def test_acc():
                return torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float())
            
            results = model.train(dataset, opt="LBFGS", steps=20, metrics=(train_acc, test_acc))
            formula = model.symbolic_formula()[0][0]
            print(formula)
            
            def acc(formula, X, y):
                batch = X.shape[0]
                correct = 0
                for i in range(batch):
                    formula_eval = formula
                    for j in range(X.size(1)):
                        index = j + 1
                        formula_eval = formula_eval.subs('x_{0}'.format(index), X[i, j].item())
                    correct += (np.round(np.array(formula_eval).astype(np.float64)) == y[i, 0].item())
                return correct / batch

            print('train acc of the formula:', acc(formula, dataset['train_input'], dataset['train_label']))
            print('test acc of the formula:', acc(formula, dataset['test_input'], dataset['test_label']))

            train_losses += results['train_loss']
            test_losses += results['test_loss']
        
    

Num Questions:  1
Selected questions:  [20] a:  0
features:  10
b:  0
Train Input Size: torch.Size([735, 10])
Train Label Size: torch.Size([735, 1])
fixing (0,0,0) with exp, r2=1.0000000000000016
fixing (0,0,1) with exp, r2=1.0000000000000016
fixing (0,0,2) with exp, r2=1.0000000000000018
fixing (0,0,3) with exp, r2=1.0000000000000013
fixing (0,0,4) with exp, r2=1.0000000000000018
fixing (0,1,0) with x^4, r2=1.000000000000002
fixing (0,1,1) with x^4, r2=1.0000000000000018
fixing (0,1,2) with exp, r2=1.0000000000000016
fixing (0,1,3) with exp, r2=1.0000000000000018
fixing (0,1,4) with exp, r2=1.0000000000000016
fixing (0,2,0) with exp, r2=1.0000000000000018
fixing (0,2,1) with x^4, r2=1.0000000000000013
fixing (0,2,2) with exp, r2=1.0000000000000016
fixing (0,2,3) with exp, r2=1.0000000000000013
fixing (0,2,4) with x^4, r2=1.0000000000000016
fixing (0,3,0) with exp, r2=1.0000000000000027
fixing (0,3,1) with exp, r2=1.0000000000000022
fixing (0,3,2) with exp, r2=1.0000000000000022
fixing

train loss: 3.78e-01 | test loss: 3.84e-01 | reg: 9.65e+01 : 100%|██| 20/20 [00:06<00:00,  3.10it/s]


0.16*(-0.05*(-x_10 - 0.43)**4 + 0.07*exp(3.66*x_7) - 0.22*sin(0.44*x_6 - 7.41) - 1)**2 - 0.49*sin(0.08*sin(0.41*x_6 - 7.19) + 1.03) - 0.7*sin(0.01*(-x_10 - 0.98)**4 + 0.03*(-x_5 - 0.66)**4 + 0.e-2*exp(4.69*x_4) - 0.16*sin(0.34*x_6 - 3.97) - 4.07) - 1.06*tanh(-0.03*(-x_2 - 0.5)**4 - 0.01*(-x_3 - 0.77)**4 + 0.02*(-x_7 - 0.88)**4 + 0.03*exp(1.89*x_5) + 0.4*sin(0.42*x_6 + 2.0) + 0.72) - 0.32*tanh(0.01*exp(5.58*x_1) - 0.e-2*exp(9.28*x_10) + 0.01*exp(4.95*x_7) + 0.01*exp(6.97*x_8) + 0.01*exp(7.65*x_9) + 0.48*sin(0.4*x_6 - 4.19) + 0.09) + 1.87
train acc of the formula: 0.6557823129251701
test acc of the formula: 0.6304347826086957
b:  1
Train Input Size: torch.Size([735, 10])
Train Label Size: torch.Size([735, 1])
fixing (0,0,0) with exp, r2=1.0000000000000018
fixing (0,0,1) with x^4, r2=1.0000000000000013
fixing (0,0,2) with exp, r2=1.0000000000000016
fixing (0,0,3) with exp, r2=1.0000000000000016
fixing (0,0,4) with exp, r2=1.0000000000000018
fixing (0,1,0) with exp, r2=1.0000000000000013
f

train loss: 3.83e-01 | test loss: 3.89e-01 | reg: 6.71e+01 : 100%|██| 20/20 [00:06<00:00,  3.24it/s]


0.16*(-0.e-2*(-1.0*x_5 - 1)**4 - 0.e-2*exp(4.42*x_10) - 0.01*exp(3.16*x_3) + 0.03*exp(3.8*x_7) + 0.02*exp(2.98*x_8) - 0.22*sin(0.44*x_6 - 7.41) - 1)**2 - 0.49*sin(0.01*exp(4.65*x_9) + 0.08*sin(0.41*x_6 - 7.19) + 1.03) - 0.7*sin(0.01*(-0.97*x_3 - 1)**4 + 0.02*(-x_5 - 0.7)**4 + 0.01*exp(2.89*x_10) + 0.03*exp(2.24*x_7) - 0.16*sin(0.35*x_6 - 3.96) - 3.74 - 0.34*exp(-7.4*x_8)) - 1.06*tanh(0.01*(-x_1 - 0.58)**4 + 0.01*exp(3.98*x_2) - 0.e-2*exp(2.8*x_3) + 0.17*exp(3.96*x_5) + 0.03*exp(2.81*x_7) + 0.4*sin(0.42*x_6 + 2.0) + 0.72) - 0.31*tanh(-0.02*(-x_2 - 0.6)**4 - 0.01*(-x_7 - 0.81)**4 + 0.02*exp(4.84*x_10) + 0.01*exp(7.46*x_3) + 0.01*exp(5.88*x_4) + 0.49*sin(0.4*x_6 - 4.19) + 0.09) + 1.87
train acc of the formula: 0.5047619047619047
test acc of the formula: 0.4782608695652174
b:  2
Train Input Size: torch.Size([735, 10])
Train Label Size: torch.Size([735, 1])
fixing (0,0,0) with exp, r2=1.0000000000000016
fixing (0,0,1) with exp, r2=1.0000000000000016
fixing (0,0,2) with x^4, r2=1.00000000000

...

0.16*(0.01*exp(4.48*x_10) + 0.01*exp(1.55*x_3) + 0.01*exp(4.63*x_7) - 0.22*sin(0.44*x_6 - 7.42) - 1)**2 - 0.68*sin(0.02*(-x_3 - 0.66)**4 + 0.01*exp(2.64*x_1) + 0.03*exp(2.11*x_5) - 0.16*sin(0.34*x_6 - 3.97) - 4.09) - 0.35*tanh(0.01*exp(8.66*x_1) + 0.03*exp(3.0*x_2) + 0.02*exp(4.74*x_5) - 0.03*exp(1.65*x_7) + 0.43*sin(0.4*x_6 - 4.19) + 0.1) - 1.03*tanh(-0.02*(-0.88*x_2 - 1)**4 + 0.04*exp(8.0*x_4) + 0.02*exp(5.32*x_5) + 0.02*exp(3.06*x_7) + 0.1*exp(2.61*x_8) + 0.05*exp(7.81*x_9) + 0.41*sin(0.42*x_6 + 2.0) + 0.72) - 0.34*tanh(0.01*exp(3.1*x_10) - 0.02*exp(7.02*x_2) + 0.02*exp(7.69*x_5) + 0.05*exp(7.43*x_7) - 0.02*exp(9.42*x_8) - 0.e-2*exp(9.03*x_9) + 0.1*sin(0.4*x_6 - 7.19) + 0.77) + 1.63
train acc of the formula: 0.41496598639455784
test acc of the formula: 0.358695652173913
b:  8
Train Input Size: torch.Size([735, 10])
Train Label Size: torch.Size([735, 1])
fixing (0,0,0) with exp, r2=1.0000000000000016
fixing (0,0,1) with exp, r2=1.0000000000000018
fixing (0,0,2) with exp, r2=1.0000000

train loss: 4.40e-01 | test loss: 4.04e-01 | reg: 7.93e+01 : 100%|██| 20/20 [00:06<00:00,  3.08it/s]


0.17*(-0.01*(-x_5 - 0.85)**4 + 0.01*(-0.91*x_8 - 1)**4 - 0.22*sin(0.44*x_6 - 7.42) - 1)**2 - 0.68*sin(0.03*(-x_10 - 0.71)**4 + 0.02*(-x_3 - 0.56)**4 + 0.01*exp(3.96*x_2) + 0.07*exp(1.75*x_5) + 0.01*exp(2.34*x_9) - 0.16*sin(0.35*x_6 - 3.97) - 3.78 - 0.35*exp(-6.43*x_8)) - 0.34*tanh(-0.05*(-x_10 - 0.66)**4 + 0.03*exp(2.2*x_1) + 0.04*exp(3.2*x_7) + 0.1*sin(0.41*x_6 - 7.19) + 0.74) + 0.34*tanh(0.01*(-0.97*x_7 - 1)**4 + 0.02*exp(6.6*x_1) + 0.01*exp(2.88*x_2) + 0.07*exp(5.0*x_5) - 0.44*sin(0.4*x_6 - 4.19) - 0.09) - 1.03*tanh(-0.01*(-x_2 - 0.91)**4 + 0.01*exp(6.69*x_10) + 0.01*exp(8.66*x_4) + 0.01*exp(9.43*x_5) + 0.01*exp(5.81*x_7) - 0.02*exp(3.68*x_8) + 0.41*sin(0.42*x_6 + 2.0) + 0.7) + 1.64
train acc of the formula: 0.6
test acc of the formula: 0.6413043478260869
b:  9
Train Input Size: torch.Size([735, 10])
Train Label Size: torch.Size([735, 1])
fixing (0,0,0) with x^4, r2=1.0000000000000013
fixing (0,0,1) with exp, r2=1.0000000000000016
fixing (0,0,2) with x^4, r2=1.0000000000000016
fixin

train loss: 4.38e-01 | test loss: 4.14e-01 | reg: 2.49e+02 : 100%|██| 20/20 [00:06<00:00,  3.03it/s]


0.17*(0.02*(-x_1 - 0.99)**4 - 0.01*(-x_2 - 0.66)**4 - 0.02*(-x_3 - 0.55)**4 + 0.05*exp(3.71*x_8) + 0.03*exp(4.0*x_9) - 0.22*sin(0.44*x_6 - 7.42) - 1)**2 + 0.68*sin(0.01*exp(4.12*x_2) - 0.08*exp(1.65*x_8) + 0.16*sin(0.35*x_6 - 3.97) + 4.14) - 0.35*tanh(-0.02*(-x_2 - 0.78)**4 - 0.01*(-0.84*x_8 - 1)**4 + 0.e-2*exp(4.69*x_1) + 0.e-2*exp(3.76*x_10) + 0.43*sin(0.4*x_6 - 4.19) + 0.07) - 0.34*tanh(0.03*(-x_1 - 0.89)**4 - 0.04*(-x_10 - 0.87)**4 - 0.03*exp(3.21*x_2) + 0.09*exp(4.82*x_4) + 0.01*exp(3.8*x_7) - 0.06*exp(9.44*x_9) + 0.1*sin(0.41*x_6 - 7.2) + 0.77) - 1.03*tanh(0.01*(-x_5 - 0.62)**4 + 0.02*exp(4.81*x_10) - 0.02*exp(2.48*x_2) + 0.03*exp(9.43*x_4) + 0.16*exp(5.02*x_7) + 0.4*exp(3.2*x_8) + 0.41*sin(0.42*x_6 + 2.0) + 0.72) + 1.63
train acc of the formula: 0.3904761904761905
test acc of the formula: 0.34782608695652173
